In [1]:
#installing transformes
!pip install datasets transformers seqeval

In [2]:
!pip install evaluate

In [3]:
import evaluate
accuracy = evaluate.load("accuracy")

In [4]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
#reward_name = "OpenAssistant/reward-model-deberta-v3-large"
model_name = 'microsoft/deberta-base'
model, tokenizer = AutoModelForSequenceClassification.from_pretrained(model_name), AutoTokenizer.from_pretrained(model_name,use_fast=True)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.weight', 'classi

In [6]:
from datasets import DatasetDict, Dataset,load_metric
import tensorflow as tf


In [7]:
df = pd.read_csv(r'/content/gdrive/MyDrive/patent project/clean_data_with_enrich.csv',dtype={'filing_year':'str'})

<ipython-input-7-7ba450f1f037>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/content/gdrive/MyDrive/patent project/clean_data_with_enrich.csv',dtype={'filing_year':'str'})


In [8]:
df = df.loc[df['one_if_small']==1,:]

In [9]:
df = df[['clean_abstract','Gender_Target']]

In [10]:
df = df.rename(columns={"Gender_Target": "label"})

In [11]:
df['label'].value_counts()

Male      124154
Female     18637
Name: label, dtype: int64

In [12]:
def preprocess_function(data):
      return tokenizer(data, truncation=True)

In [13]:
dataset = Dataset.from_pandas(df)


In [14]:
label_map = {"Male": 0, "Female": 1}
dataset = dataset.map(lambda example: {"label": label_map[example["label"]]})

Map:   0%|          | 0/142791 [00:00<?, ? examples/s]

In [15]:
encode_dataset = dataset.map(lambda x: preprocess_function(x['clean_abstract']))

Map:   0%|          | 0/142791 [00:00<?, ? examples/s]

In [16]:
encode_dataset = encode_dataset.train_test_split(0.2, seed=42)

In [17]:
minority_indices = [i for i in range(len(encode_dataset["train"])) if encode_dataset["train"][i]["label"] == 1]
majority_indices = [i for i in range(len(encode_dataset["train"])) if encode_dataset["train"][i]["label"] == 0]


In [18]:
len(majority_indices)

99310

In [19]:
import numpy as  np
oversampled_minority_indices = np.random.choice(minority_indices, size=99310, replace=True)


In [20]:
balanced_indices = np.concatenate([oversampled_minority_indices, majority_indices])


In [21]:
np.random.shuffle(balanced_indices)


In [22]:
encode_dataset["train"] = encode_dataset["train"].select(balanced_indices)


In [23]:
minority_indices = [i for i in range(len(encode_dataset["test"])) if encode_dataset["test"][i]["label"] == 1]
majority_indices = [i for i in range(len(encode_dataset["test"])) if encode_dataset["test"][i]["label"] == 0]


In [24]:
len(minority_indices)

3715

In [25]:
under_sampled_majority_indices = np.random.choice(majority_indices, size=3715, replace=True)


In [26]:
balanced_indices_test = np.concatenate([under_sampled_majority_indices, minority_indices])


In [27]:
np.random.shuffle(balanced_indices_test)


In [28]:
encode_dataset["test"] = encode_dataset["test"].select(balanced_indices_test)


In [29]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [30]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2,ignore_mismatched_sizes=True)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.weight', 'classi

In [31]:
metric = load_metric('accuracy')

<ipython-input-31-343a7ea47872>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [32]:
!pip install accelerate -U


In [33]:
!pip install transformers[torch]

In [34]:
args = TrainingArguments(
    '/content/gdrive/MyDrive/patent project/',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=0.000002,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy')
    #push_to_hub=True,

In [35]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = metric.compute(predictions=predictions, references=labels)
    return acc

In [36]:
trainer = Trainer(
    model,
    args,
    train_dataset=encode_dataset["train"],
    eval_dataset=encode_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

In [37]:
with tf.device('/device:GPU:0'):
  trainer.train()
  trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.528700,0.638221,0.693943
2,0.463300,1.085184,0.681427
3,0.507600,1.727316,0.666487


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2020 in _inner_training_loop     │
│                                                                                                  │
│   2017 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   2018 │   │   │   │   │   self.control = self.callback_handler.on_step_end(args, self.state, s  │
│   2019 │   │   │   │   │                                                                         │
│ ❱ 2020 │   │   │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_k  │
│   2021 │   │   │   │   else:                                                                     │
│   2022 │   │   │   │   │   self.control = self.callback_handler.on_substep_end(args, self.state  │
│   2023                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2307 in _maybe_log_save_evaluate │
│                                                                                                  │
│   2304 │   │   │   self._globalstep_last_logged = self.state.global_step                         │
│   2305 │   │   │   self.store_flos()                                                             │
│   2306 │   │   │                                                                                 │
│ ❱ 2307 │   │   │   self.log(logs)                                                                │
│   2308 │   │                                                                                     │
│   2309 │   │   metrics = None                                                                    │
│   2310 │   │   if self.control.should_evaluate:                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2672 in log                      │
│                                                                                                  │
│   2669 │   │                                                                                     │
│   2670 │   │   output = {**logs, **{"step": self.state.global_step}}                             │
│   2671 │   │   self.state.log_history.append(output)                                             │
│ ❱ 2672 │   │   self.control = self.callback_handler.on_log(self.args, self.state, self.control,  │
│   2673 │                                                   